In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True 
session = InteractiveSession(config=config)

In [ ]:
import os
import cv2 
import sys 
import random
import warnings
import numpy as np
import pandas as pd  
from time import time 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
from itertools import chain 
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label 

import tensorflow as tf 
from tensorflow.keras.layers import Input, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, Activation, Add, multiply, add, concatenate, LeakyReLU, ZeroPadding2D, UpSampling2D, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K 

from sklearn.metrics import classification_report, confusion_matrix
%matplotlib inline 

IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_CHANNEL = 3
IMG_PATH = '../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/png_images'
MASK_PATH = '../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/png_masks'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42 
random.seed = seed
np.random.seed = seed

In [ ]:
train_df = pd.read_csv('../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/stage_1_train_images.csv')
test_df = pd.read_csv('../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/stage_1_test_images.csv')

In [ ]:
train_df_pneumo = train_df[train_df['has_pneumo'] == 1]

In [ ]:
print("Getting and Resizing Train Images, Train Mask, and Adding Label ...\n\n")

# Create X_train, Y_train, and Label
X_train_seg = np.zeros((len(train_df_pneumo), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL), dtype=np.uint8)
Y_train_seg = np.zeros((len(train_df_pneumo), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
Label_seg = np.zeros(len(train_df_pneumo), dtype=np.uint8)

img_data = list(train_df_pneumo.T.to_dict().values())

for i, data_row in tqdm(enumerate(img_data), total=len(img_data)):
    
    patientImage = data_row['new_filename']
    imageLabel  = data_row['has_pneumo']

    imagePath = os.path.join(IMG_PATH, patientImage)
    lungImage = imread(imagePath)
    lungImage = np.expand_dims(resize(lungImage, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True), axis=-1)
    
    X_train_seg[i] = lungImage

    Label_seg[i] = imageLabel

    maskPath = os.path.join(MASK_PATH, patientImage)
    maskImage = imread(maskPath)
    maskImage = np.expand_dims(resize(maskImage, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True), axis=-1)

    Y_train_seg[i] = maskImage

print('\n\nProcess ... C O M P L E T E')

In [ ]:
# Illustrate the train images and masks
plt.figure(figsize=(20, 16))
x, y = 12, 4
for i in range(y):
    for j in range(x):
        plt.subplot(y*2, x, i*2*x+j+1)
        pos = i*120 + j*10
        plt.imshow(X_train_seg[pos], cmap=plt.cm.bone)
        plt.title('Image #{}'.format(pos))
        plt.axis('off')
        plt.subplot(y*2, x, (i*2+1)*x+j+1)

        plt.imshow(np.squeeze(Y_train_seg[pos]), cmap='gray_r')
        plt.title('Mask #{}\nLabel: {}'.format(pos, Label_seg[pos]))
        plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def iou_loss_score(y_pred, y_true, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true, -1) + K.sum(y_pred, -1) - intersection
    iou = (intersection + smooth)/(union + smooth)
    return iou

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32')
    intersection = y_true_f * y_pred_f
    score = 2. * K.sum(intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return score

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def bce_logdice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) - K.log(1. - dice_loss(y_true, y_pred))

In [ ]:
def UNetPlusPlus(num_class=1, deep_supervision=False):
    nb_filter = [32, 64, 128, 256, 512]
    img_rows = IMG_HEIGHT
    img_cols = IMG_WIDTH
    color_type = 3
    bn_axis = 3

    img_input = Input(shape=(img_rows, img_cols, color_type), name='main_input')

    conv1_1 = Conv2D(nb_filter[0], (3, 3), activation='relu', padding='same')(img_input)
    conv1_1 = Conv2D(nb_filter[0], (3, 3), activation='relu', padding='same')(conv1_1)
    pool1 = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(conv1_1)

    conv2_1 = Conv2D(nb_filter[1], (3, 3), activation='relu', padding='same')(pool1)
    conv2_1 = Conv2D(nb_filter[1], (3, 3), activation='relu', padding='same')(conv2_1)
    pool2 = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(conv2_1)

    up1_2 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up12', padding='same')(conv2_1)
    conv1_2 = concatenate([up1_2, conv1_1], name='merge12', axis=bn_axis)
    conv1_2 = Conv2D(nb_filter[0], (3, 3), activation='relu', padding='same')(conv1_2)
    conv1_2 = Conv2D(nb_filter[0], (3, 3), activation='relu', padding='same')(conv1_2)

    conv3_1 = Conv2D(nb_filter[2], (3, 3), activation='relu', padding='same')(pool2)
    conv3_1 = Conv2D(nb_filter[2], (3, 3), activation='relu', padding='same')(conv3_1)
    pool3 = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(conv3_1)

    up2_2 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up22', padding='same')(conv3_1)
    conv2_2 = concatenate([up2_2, conv2_1], name='merge22', axis=bn_axis)
    conv2_2 = Conv2D(nb_filter[1], (3, 3), activation='relu', padding='same')(conv2_2)
    conv2_2 = Conv2D(nb_filter[1], (3, 3), activation='relu', padding='same')(conv2_2)

    up1_3 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up13', padding='same')(conv2_2)
    conv1_3 = concatenate([up1_3, conv1_1, conv1_2], name='merge13', axis=bn_axis)
    conv1_3 = Conv2D(nb_filter[0], (3, 3), activation='relu', padding='same')(conv1_3)
    conv1_3 = Conv2D(nb_filter[0], (3, 3), activation='relu', padding='same')(conv1_3)

    conv4_1 = Conv2D(nb_filter[3], (3, 3), activation='relu', padding='same')(pool3)
    conv4_1 = Conv2D(nb_filter[3], (3, 3), activation='relu', padding='same')(conv4_1)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(conv4_1)

    up3_2 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up32', padding='same')(conv4_1)
    conv3_2 = concatenate([up3_2, conv3_1], name='merge32', axis=bn_axis)
    conv3_2 = Conv2D(nb_filter[2], (3, 3), activation='relu', padding='same')(conv3_2)
    conv3_2 = Conv2D(nb_filter[2], (3, 3), activation='relu', padding='same')(conv3_2)

    up2_3 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up23', padding='same')(conv3_2)
    conv2_3 = concatenate([up2_3, conv2_1, conv2_2], name='merge23', axis=bn_axis)
    conv2_3 = Conv2D(nb_filter[1], (3, 3), activation='relu', padding='same')(conv2_3)
    conv2_3 = Conv2D(nb_filter[1], (3, 3), activation='relu', padding='same')(conv2_3)

    up1_4 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up14', padding='same')(conv2_3)
    conv1_4 = concatenate([up1_4, conv1_1, conv1_2, conv1_3], name='merge14', axis=bn_axis)
    conv1_4 = Conv2D(nb_filter[0], (3, 3), activation='relu', padding='same')(conv1_4)
    conv1_4 = Conv2D(nb_filter[0], (3, 3), activation='relu', padding='same')(conv1_4)

    # conv5_1 = standard_unit(pool4, stage='51', nb_filter=nb_filter[4])
    conv5_1 = Conv2D(nb_filter[4], (3, 3), activation='relu', padding='same')(pool4)
    conv5_1 = Conv2D(nb_filter[4], (3, 3), activation='relu', padding='same')(conv5_1)

    up4_2 = Conv2DTranspose(nb_filter[3], (2, 2), strides=(2, 2), name='up42', padding='same')(conv5_1)
    conv4_2 = concatenate([up4_2, conv4_1], name='merge42', axis=bn_axis)
    conv4_2 = Conv2D(nb_filter[3], (3, 3), activation='relu', padding='same')(conv4_2)
    conv4_2 = Conv2D(nb_filter[3], (3, 3), activation='relu', padding='same')(conv4_2)

    up3_3 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up33', padding='same')(conv4_2)
    conv3_3 = concatenate([up3_3, conv3_1, conv3_2], name='merge33', axis=bn_axis)
    conv3_3 = Conv2D(nb_filter[2], (3, 3), activation='relu', padding='same')(conv3_3)
    conv3_3 = Conv2D(nb_filter[2], (3, 3), activation='relu', padding='same')(conv3_3)

    up2_4 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up24', padding='same')(conv3_3)
    conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], name='merge24', axis=bn_axis)
    conv2_4 = Conv2D(nb_filter[1], (3, 3), activation='relu', padding='same')(conv2_4)
    conv2_4 = Conv2D(nb_filter[1], (3, 3), activation='relu', padding='same')(conv2_4)

    up1_5 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up15', padding='same')(conv2_4)
    conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], name='merge15', axis=bn_axis)
    conv1_5 = Conv2D(nb_filter[0], (3, 3), activation='relu', padding='same')(conv1_5)
    conv1_5 = Conv2D(nb_filter[0], (3, 3), activation='relu', padding='same')(conv1_5)

    nestnet_output_1 = Conv2D(num_class, (1, 1), activation='sigmoid', name='output_1', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_2)
    nestnet_output_2 = Conv2D(num_class, (1, 1), activation='sigmoid', name='output_2', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_3)
    nestnet_output_3 = Conv2D(num_class, (1, 1), activation='sigmoid', name='output_3', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_4)
    nestnet_output_4 = Conv2D(num_class, (1, 1), activation='sigmoid', name='output_4', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_5)

    model = Model(inputs=img_input, outputs=[nestnet_output_4])

    return model

In [ ]:
model = UNetPlusPlus()

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=[bce_dice_loss, iou_loss_score]
)

In [ ]:
reduceLROnPlat = ReduceLROnPlateau(monitor='val_bce_dice_loss', factor=0.20, min_delta=1e-15 ,patience=3, 
                                   verbose=1, mode='auto')

callbacks = [reduceLROnPlat]

In [ ]:
epochs = 100
model.fit(X_train_seg, Y_train_seg, validation_split=0.1, batch_size=16, epochs=epochs, callbacks=callbacks)

In [ ]:
# The first 90% used for training
pred_train = model.predict(X_train_seg[:int(X_train_seg.shape[0]*0.9)].astype(np.float16), verbose=1)
# The last 10% used for validation
pred_val = model.predict(X_train_seg[int(X_train_seg.shape[0]*0.9):].astype(np.float16), verbose=1)

# pred_test = model.predict(X_test, verbose=1)

# Thresholds prediction
pred_train_threshold = (pred_train > 0.4).astype(np.float16)
pred_val_threshold = (pred_val > 0.4).astype(np.float16)

In [ ]:
## Showing our predicted masks on our training data
ix = random.randint(0, 682)
plt.figure(figsize=(20, 28))

# Our original training image
plt.subplot(131)
imshow(X_train_seg[ix])
plt.title('Image')

# Our original combined mask
plt.subplot(132)
imshow(np.squeeze(Y_train_seg[ix]), cmap='gray_r')
plt.title('Mask')

# The mask of our model U-Net prediction
plt.subplot(133)
imshow(np.squeeze(pred_train_threshold[ix] > 0.4), cmap='gray_r')
plt.title('Prediction')
plt.show()